In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [39]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
sample_solution = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')

In [40]:
sample_solution.head()

In [41]:
train.head()

In [42]:
test.head()

In [43]:
data = pd.concat([train, test], sort=False)

In [44]:
data.head()

In [45]:
print(len(train), len(test), len(data))

In [46]:
data.isnull().sum()

In [47]:
data.isnull().sum() / len(data) * 100

In [48]:
for i in test.columns[1 : ]:
    i_avg = data[i].mean()
    i_std = data[i].std()
    data[i].fillna(np.random.uniform(i_avg - i_std, i_avg + i_std), inplace = True)

In [49]:
train = data[ : len(train)]
test = data[len(train) : ]

In [50]:
y_train = train['claim']
X_train = train.drop('claim', axis = 1)
X_test = test.drop('claim', axis = 1)

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.3, random_state = 0, stratify = y_train)

In [56]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid)

In [59]:
params = {
    'objective': 'binary'
}

model = lgb.train(params, lgb_train,
                  valid_sets = [lgb_train, lgb_eval],
                  verbose_eval = 10,
                  num_boost_round = 1000,
                  early_stopping_rounds = 10)

y_pred = model.predict(X_test, num_iteration = model.best_iteration)

In [61]:
y_pred = (y_pred > 0.5).astype(int)
sub = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')
sub['claim'] = y_pred
sub.to_csv('submission_lightgbm.csv', index = False)